In [ ]:
import cv2
import os
import numpy as np
from mtcnn.mtcnn import MTCNN
from pathlib import Path
import warnings



input_base_dir = Path(r"E:\FakeAvCeleb\FakeAVCeleb_v1.2\FakeAVCeleb_v1.2")

print(f"Checking if input directory exists: {input_base_dir}")
print(f"Input directory exists: {input_base_dir.exists()}")
if input_base_dir.exists():
    print("Contents of input directory:")
    for item in input_base_dir.iterdir():
        print(f"- {item.name}")

output_base_dir = Path(r"E:\Processed_FakeAVCeleb")

#Definine the source folders and their corresponding output labels (0=real, 1=fake)
SOURCE_FOLDERS = [
    ("RealVideo-RealAudio", "0_real"),
    ("FakeVideo-RealAudio", "1_fake")
]

# List of video file extensions
video_extensions = {".mp4", ".avi", ".mov", ".mkv", ".flv", ".wmv"}

# --- Parameters ---
motion_threshold = 0.2 # Threshold for motion detection originaly was 0.7
output_face_size = (256, 256)


MIN_FRAMES_PER_VIDEO = 24 
MAX_FRAMES_PER_VIDEO = 24 

# --- Initializations ---
detector = MTCNN()
warnings.filterwarnings('ignore', category=FutureWarning) # Suppress MTCNN FutureWarnings
print("Setup complete. Ready to process FakeAVCeleb.")

Checking if input directory exists: E:\FakeAvCeleb\FakeAVCeleb_v1.2\FakeAVCeleb_v1.2
Input directory exists: True
Contents of input directory:
- FakeVideo-FakeAudio
- FakeVideo-RealAudio
- meta_data.csv
- README.txt
- RealVideo-RealAudio
Setup complete. Ready to process FakeAVCeleb.
Setup complete. Ready to process FakeAVCeleb.


In [ ]:
# Function to sample frames based on optical flow 
def sample_frames_with_optical_flow(video_path, threshold):
    
    cap = cv2.VideoCapture(str(video_path))
    if not cap.isOpened():
        print(f"Error: Could not open video file at {video_path}")
        return []
    
    selected_frames = []
    #Reading the first frame and conversion to grayscale

    ret, prev_frame = cap.read() #ret= True if frame is read correctly
    if not ret:
        print(f"Error: Could not read the first frame of {video_path}")
        return []
        
    prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    
    selected_frames.append(prev_frame) # Adding the first frame to our list by default

    while True:
        # Reading the next frame
        ret, frame = cap.read()
        if not ret:
            break # End of video
        
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        # Calculating dense optical flow between previous and current frame
        flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0) #Dense optical flow is calculated using Farneback method which gives vector field representing motion between two frames
        
        # Calculate the magnitude of the flow vectors and angle so that we can assess motion
        magnitude, angle = cv2.cartToPolar(flow[..., 0], flow[..., 1]) #cv2.cartToPolar computes the magnitude and angle of 2D vectors
        
        # Calculating the average motion score for the frame
        mean_magnitude = np.mean(magnitude)
        
        # If motion is above the threshold, keep the frame
        if mean_magnitude > threshold:
            selected_frames.append(frame)
        
        prev_gray = gray

    # Release the video capture object
    cap.release()
    return selected_frames

In [ ]:
def align_and_crop_faces(frames, size): #Detects, aligns, and crops faces from a list of frames using MTCNN.
    
    processed_faces = []
    
    for frame in frames:
        # MTCNN expects RGB format so convert BGR to RGB
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Detect faces
        results = detector.detect_faces(frame_rgb)
        
        #  We only process  if one confident face is found
        if len(results) == 1:
            keypoints = results[0]['keypoints']
            left_eye = keypoints['left_eye']
            right_eye = keypoints['right_eye']
            
            #Alignment is done based on eye positions 
            dY = right_eye[1] - left_eye[1] #Vertical distance between eyes
            dX = right_eye[0] - left_eye[0] #Horizontal distance between eyes
            angle = np.degrees(np.arctan2(dY, dX)) #Calculate angle between eyes in degrees
            
            (h, w) = frame.shape[:2] #Getting frame dimensions
            center = (w // 2, h // 2)
            
            M = cv2.getRotationMatrix2D(center, angle, 1.0) #Get rotation matrix for rotating the image around its center by the calculated angle
            rotated_frame = cv2.warpAffine(frame, M, (w, h), flags=cv2.INTER_CUBIC) #Rotating the frame to align eyes horizontally
            #cv2.warpaffine works by applying an affine transformation to the image using the rotation matrix M
            
            #  Re-detecting face for Accurate Cropping 
            rotated_frame_rgb = cv2.cvtColor(rotated_frame, cv2.COLOR_BGR2RGB)
            new_results = detector.detect_faces(rotated_frame_rgb)
            
            if len(new_results) == 1:
                #Get bounding box of the detected face
                x, y, width, height = new_results[0]['box']
                # Add boundary checks for safety to ensure we don't go out of image bounds
                y1, y2 = max(0, y), min(rotated_frame.shape[0], y + height)
                x1, x2 = max(0, x), min(rotated_frame.shape[1], x + width)
                #Cropping the face from the rotated frame
                face_crop = rotated_frame[y1:y2, x1:x2]
                #Only process if the cropped face is valid
                if face_crop.size > 0:
                    resized_face = cv2.resize(face_crop, size, interpolation=cv2.INTER_AREA)
                    processed_faces.append(resized_face)
                    
    return processed_faces

In [4]:
def save_processed_frames(faces, output_folder, unique_video_name):
    """
    Saves a list of face images to a target directory.
    Each video's frames will be in its own subdirectory.
    
    Args:
        faces (list): A list of processed face images (numpy arrays).
        output_folder (Path): The base output directory (e.g., .../0_real or .../1_fake).
        unique_video_name (str): The unique name for the video folder (e.g., "id00076_00109_1").
    """
    # Create a specific folder for this video's frames
    video_output_folder = output_folder / unique_video_name
    video_output_folder.mkdir(parents=True, exist_ok=True)
    
    # Save each frame in the video-specific folder
    for i, face in enumerate(faces):
        filename = f"{i:04d}.png"
        save_path = video_output_folder / filename
        cv2.imwrite(str(save_path), face)

In [ ]:
# --- Main processing loop for FakeAVCeleb (with Frame Limiting and resume support) ---

print("Starting FakeAVCeleb processing...")
print(f"Frame selection policy: MIN {MIN_FRAMES_PER_VIDEO}, MAX {MAX_FRAMES_PER_VIDEO} (based on motion threshold {motion_threshold})")

# Counters for final summary
total_videos_processed = 0
total_videos_skipped = 0
total_videos_already = 0

for source_folder, label in SOURCE_FOLDERS:
    # e.g., source_folder = "RealVideo-RealAudio", label = "0_real"
    
    input_dir = input_base_dir / source_folder
    # The output_dir is now the final destination folder, e.g., E:\Processed_FakeAVCeleb\0_real
    output_dir = output_base_dir / label
    
    if not input_dir.exists():
        print(f"Warning: Source directory not found. Skipping: {input_dir}")
        continue
        
    print(f"--- Processing source folder: {source_folder} (Saving to {output_dir}) ---")
    
    # Iterate through Ethnicity folders (e.g., "African", "Asian (East)")
    for ethnicity_dir in input_dir.iterdir():
        if not ethnicity_dir.is_dir():
            continue
            
        # Iterate through Gender folders (e.g., "men", "women")
        for gender_dir in ethnicity_dir.iterdir():
            if not gender_dir.is_dir():
                continue
                
            # Iterate through ID folders (e.g., "id00076")
            for id_dir in gender_dir.iterdir():
                if not id_dir.is_dir():
                    continue
                    
                # We are now in the folder containing the videos
                # (e.g., .../FakeVideo-RealAudio/African/men/id00076/)
                
                # Iterate through all video files in this folder
                for video_file in id_dir.iterdir():
                    # Check if it's a video file we can process
                    if video_file.is_file() and video_file.suffix.lower() in video_extensions:

                        # Build the unique video name and output folder for resume checks
                        unique_video_name = f"{id_dir.name}_{video_file.stem}"
                        video_output_folder = (output_dir / unique_video_name)

                        # If video_output_folder already exists and contains enough frames, skip it
                        if video_output_folder.exists():
                            existing_files = list(video_output_folder.glob('*.png'))
                            if len(existing_files) >= MIN_FRAMES_PER_VIDEO:
                                print(f"Already processed, skipping: {video_file}")
                                total_videos_already += 1
                                continue

                        # Print the full path of the video being processed so you can see progress
                        print(f"Processing video: {video_file}")

                        # Step 1: Sample frames
                        frames = sample_frames_with_optical_flow(video_file, motion_threshold)
                        
                        # --- NEW: Check MIN frame count ---
                        if len(frames) < MIN_FRAMES_PER_VIDEO:
                            print(f"Skipping {video_file.name}: only {len(frames)} frames. (Min: {MIN_FRAMES_PER_VIDEO})")
                            total_videos_skipped += 1
                            continue # Skip this video
                            
                        # Step 2: Align faces
                        faces = align_and_crop_faces(frames, output_face_size)
                        
                        # --- NEW: Check MIN face count (in case some frames fail detection) ---
                        if len(faces) < MIN_FRAMES_PER_VIDEO:
                            print(f"Skipping {video_file.name}: only {len(faces)} faces detected. (Min: {MIN_FRAMES_PER_VIDEO})")
                            total_videos_skipped += 1
                            continue # Skip this video
                        
                        # --- NEW: Apply MAX frame cap ---
                        if len(faces) > MAX_FRAMES_PER_VIDEO:
                            faces = faces[:MAX_FRAMES_PER_VIDEO] # Take only the first MAX_FRAMES
                        
                        # Step 3: Save frames in per-video folder
                        save_processed_frames(faces, output_dir, unique_video_name)
                        print(f"Saved {len(faces)} faces for {unique_video_name} -> {video_output_folder}")
                        total_videos_processed += 1

print("\nAll processing complete.")
print(f"Total videos processed and saved: {total_videos_processed}")
print(f"Total videos skipped (insufficient frames): {total_videos_skipped}")
print(f"Total videos already processed and skipped: {total_videos_already}")

Starting FakeAVCeleb processing...
Frame selection policy: MIN 5, MAX 12 (based on motion threshold 0.7)
--- Processing source folder: RealVideo-RealAudio (Saving to E:\Processed_FakeAVCeleb\0_real) ---
--- Processing source folder: FakeVideo-RealAudio (Saving to E:\Processed_FakeAVCeleb\1_fake) ---
--- Processing source folder: FakeVideo-RealAudio (Saving to E:\Processed_FakeAVCeleb\1_fake) ---


KeyboardInterrupt: 